Ha: Work done so far: change all data types, drop some corrupted rows with at start_time and end_time
Delete all rows with null value.
Calculating idle time (station-level and trip-level)

Import weather data


In [ ]:
# import all relevant libraries
import pandas as pd

import numpy as np
import matplotlib.pyplot as plt
from pandas.plotting import register_matplotlib_converters

register_matplotlib_converters()
import seaborn as sns

sns.set()
sns.set_style("white")
sns.set_palette("GnBu_d")

# also import these "new" libraries
# Note: you may have to download an add them to your environment (using e.g. 'conda install -c conda-forge folium')
import folium
from folium import plugins
from folium.plugins import HeatMap
from datetime import datetime  # for working with times objects
from datetime import timedelta  # for working with times objects
import math
import random

## Step 1: Import Data 

In [ ]:
# load csv file and set the index column by 0
df = pd.read_csv("baywheels_2019", encoding="ISO-8859-1", index_col=0)
df.head(20)

In [ ]:
# take a look at the data types and general information
print(df.info())
print(df.describe())

We can see that the data types are not specified which makes it hard to work with. After inspecting the dataset, there are rows containing "Failure" so we want to remove all of these. 

In [ ]:
# create a new data frame that containts all failure rows from dft
rent_failure = df["start_time"].str.contains("Failure")
data_failure = df[rent_failure]
data_failure

In [ ]:
# remove all rows that match failure rows
df = df.drop(data_failure.index, axis=0)

## Step 2: Transform Data

Now we have to convert every feature to the correct data types in order to to make calculations and analysis.

In [ ]:
df["bike_id"]= pd.to_numeric(df["bike_id"], errors='coerce')
df["bike_id"]= df["bike_id"].astype('Int64')
df["start_station_lat"]= pd.to_numeric(df["start_station_lat"], errors='coerce')
df["start_station_lon"]=pd.to_numeric(df["start_station_lon"], errors='coerce')
df['start_station_lat'] = df['start_station_lat'].round(6)
df['start_station_lon'] = df['start_station_lon'].round(6)
df["end_station_lat"]=pd.to_numeric(df["end_station_lat"], errors='coerce')
df["end_station_lon"]=pd.to_numeric(df["end_station_lon"], errors='coerce')
df["start_station_id"] =pd.to_numeric(df["start_station_id"], errors='coerce')
df["start_station_id"]= df["start_station_id"].astype("Int64")
df["end_station_id"] =pd.to_numeric(df["end_station_id"], errors='coerce')
df["end_station_id"]= df["end_station_id"].astype("Int64")

df["start_time"]= pd.to_datetime(df["start_time"],format ='%Y-%m-%d %H:%M:%S')
df["end_time"]= pd.to_datetime(df["end_time"],format ='%Y-%m-%d %H:%M:%S')
print(df.info())

In [ ]:
# After transforming the data we check if there are any non-defined values.
df.isna().sum()

In [ ]:
df[df["start_station_id"].isnull() | df["end_station_id"].isnull()]

## Step 3: Handle missing and incorrect values

Firstly, we noticed that there are entries which have invalid start or end time (not in 2019). Since the amount of it is less than 5%, we can remove these.

In [ ]:
# drop some rows with start_time in 2013
df.drop(df[(df['start_time'].dt.year < 2018) | (df['start_time'].dt.year > 2020)].index, inplace=True)
df

Secondly, there are missing values in <b>start_station_id</b> and <b>end_station_id</b> column. We tried different approaches to handle these values:
- Using <b>k-means clustering algorithm</b> to assign an ID to the given longtitudes and latitudes.

- <b>Dropping</b> all rows with missing values.

The cost of performing the first method was higher than the latter and the amount of missing values take up less than 5%, so we decided to opt for the second approach.

In [ ]:
df.drop(df[df['start_station_id'].isnull() | df['end_station_id'].isnull()].index, inplace = True)

In [ ]:
df.hist(bins = 12, figsize = (20, 10))

Something can be interpreted from the above graph:
the stations with the station_id under 150 are more occupied than the rest, maybe they are located within city center?




Now let's try to calculate idle time
2 approach: idle time group by bike_id, and idle time group by station_id

the first one is easier to implement 

Let's go with the first approach first


In [ ]:
start_stations =df.loc[:,['start_time', 'start_station_id', 'bike_id']]
start_stations["rented"] = 1
start_stations.rename(columns = {"start_time": "timestamp", 'start_station_id': "station_id"}, inplace=True)
start_stations


Now we want to try the second approach.

In [ ]:
end_stations =df.loc[:,['end_time', 'end_station_id', 'bike_id']]
end_stations["rented"] = 0
end_stations.rename(columns={"end_time": "timestamp", 'end_station_id': "station_id"}, inplace=True)
end_stations

In [ ]:
df_merged = pd.concat([start_stations, end_stations], axis = 0)
df_merged = df_merged.sort_values(by = ["station_id", "timestamp"])
df_merged['idle_time'] = pd.Timedelta(0)
df_merged

We define a function that calculates idle time with the help of a nested loop inside the dataframe

In [ ]:


def idle_cal(df):
    
    df = df.groupby('station_id')
    results = []  # List to store the modified groups
    for group_name, group_data in df:
        group_data.sort_values(by = 'timestamp', inplace = True)
        group_data.reset_index(drop=True, inplace=True)
    
        group_data['prev_row'] = group_data.groupby("station_id")['timestamp'].shift(1)
        for i, row in group_data.iterrows():
            if i != 0:
                if row['rented'] == 1:
                    idle_time = pd.Timedelta(0)
                    #print('idle at i ', i , ':', idle_time)
                    j = i - 1 
                    while j>=1 and group_data.at[j,'rented'] == 0:
                        idle_time = group_data.at[i,'timestamp'] -  group_data.at[j,'timestamp']
                        group_data.at[j,'idle_time'] = idle_time
                        #print('idle at j ', j , ':',  idle_time)
                        j -= 1
                    
        results.append(group_data)

    modified_df = (pd.concat(results, ignore_index=True))
    df = pd.DataFrame(modified_df)
    return df
    
                    
        

              


In [ ]:
df_merged = idle_cal(df_merged)
df_merged.head(100)

In [ ]:
df_merged.to_csv('idle_time_calculated')  

In [ ]:
notnull =df_merged[df_merged['idle_time']!=pd.Timedelta(0)]
print(notnull)

In [ ]:
df_w_idle_time = pd.read_csv("idle_time_calculated")
idle_time_per_trip = df_w_idle_time[df_w_idle_time['rented'] == 0]


In [ ]:
df_w_idle_time.info()

In [ ]:
df_w_idle_time['timestamp']= pd.to_datetime(df_w_idle_time['timestamp'],format ='%Y-%m-%d %H:%M:%S')

In [ ]:
idle_time_per_trip['end_time'] = idle_time_per_trip['timestamp']
idle_time_per_trip['end_time'] = pd.to_datetime(idle_time_per_trip['end_time'], format = '%Y-%m-%d %H:%M:%S')

In [ ]:

df_trip = pd.merge(df, idle_time_per_trip, on = ['end_time', 'bike_id'])
df_trip = df_trip.sort_values(by = 'end_time')
#[df_trip['end_time'] < '2020-01-01']
df_trip.tail(20)

In [ ]:
df_trip = df_trip.drop(['Unnamed: 0','timestamp','station_id','rented'], axis=1)

df_trip.head(10)

In [ ]:

df_w_idle_time['sum_idle_time']= pd.Timedelta(0)
df_w_idle_time['idle_mode_count']= 0
df_w_idle_time['month']= df_w_idle_time['timestamp'].dt.month
df_w_idle_time.head(10)


In [ ]:
df_w_idle_time['tim'].dtypes

In [ ]:
def sum_idle_cal(df):
    
    df = df.groupby('station_id')
    results = []  # List to store the modified groups
    
    for group_name, group_data in df:
        group_data.sort_values(by = 'timestamp', inplace = True)
        group_data.reset_index(drop=True, inplace=True)
        sum_idle_time= pd.Timedelta(0)
        idle_mode_count = 0
    
        for i, row in group_data.iterrows():
            if i != 0:
                j = i - 1
                
                if row['rented']==0 and group_data.at[j,'rented'] == 1:
                    sum_idle_time += group_data.at[j,'sum_idle_time']+ group_data.at[i,'idle_time']
                    idle_mode_count +=1
                else:
                    sum_idle_time += group_data.at[j,'sum_idle_time']
                    
            else:
                sum_idle_time+= group_data.at[i,'idle_time']
                
                idle_mode_count+= 1
            print(group_name,'sum idle at i ', i , ':', sum_idle_time, 'idle count :',idle_mode_count)
            
            
                    
                    
                    
                     
                    
                    
        results.append(group_data)

    modified_df = (pd.concat(results, ignore_index=True))
    df = pd.DataFrame(modified_df)
    return df

In [59]:
 interval = timedelta(days=30) # 30-day interval
 df_w_idle_time= sum_idle_cal(df_w_idle_time)

32 days 13:46:29 idle count : 1833
27 sum idle at i  7487 : 32 days 13:46:29 idle count : 1833
27 sum idle at i  7488 : 32 days 13:46:29 idle count : 1833
27 sum idle at i  7489 : 32 days 13:46:29 idle count : 1833
27 sum idle at i  7490 : 32 days 13:46:29 idle count : 1833
27 sum idle at i  7491 : 32 days 13:46:29 idle count : 1833
27 sum idle at i  7492 : 32 days 13:51:08 idle count : 1834
27 sum idle at i  7493 : 32 days 13:51:08 idle count : 1834
27 sum idle at i  7494 : 32 days 13:51:08 idle count : 1834
27 sum idle at i  7495 : 32 days 13:51:08 idle count : 1834
27 sum idle at i  7496 : 32 days 14:02:44 idle count : 1835
27 sum idle at i  7497 : 32 days 14:02:44 idle count : 1835
27 sum idle at i  7498 : 32 days 14:02:44 idle count : 1835
27 sum idle at i  7499 : 32 days 14:05:15 idle count : 1836
27 sum idle at i  7500 : 32 days 14:05:15 idle count : 1836
27 sum idle at i  7501 : 32 days 14:05:15 idle count : 1836
27 sum idle at i  7502 : 32 days 14:08:49 idle count : 1837
27 su

KeyboardInterrupt: 

In [ ]:
df_w_idle_time.head(10)

## Step 3b: Working with Weather Data

First approaches:

In [ ]:
# import the weather data
wd = pd.read_csv("SanFrancisco", encoding="ISO-8859-1", index_col=0)
# show first 20 rows
wd.head(20)

In [ ]:
# take a look at the data types and general information
print(wd.info())
print(wd.describe())

In [ ]:
# convert timestamp to datetime
wd['timestamp'] = pd.to_datetime(wd['timestamp'],format ='%Y-%m-%d %H:%M:%S')

# extract the year from timestamp
wd['year'] = wd['timestamp'].dt.year

# count how many entries are for what year
value_counts = wd['year'].value_counts()

# show how many entries has every year, because only 2019 is important
print(value_counts)


In [ ]:
print(wd.info())

In [ ]:
# only the data for the year 2019 is important, so we drop all the other entries.
wd = wd[wd['timestamp'].dt.year == 2019]


In [ ]:
# check first 20 entries to look is it was successfull
wd.head(20)

In [ ]:
# after transforming the data we check if there are any non-defined values.
wd.isna().sum()

In [ ]:
# drop all rows with missing values
weather_2019 = wd.dropna(axis = 0)
print(weather_2019.info())
print(weather_2019.describe())

In [ ]:
# let's have a look at the temperature in Sanfrancisco in 2019
fig,ax = plt.subplots(figsize=(10,4))
ax.plot(weather_2019["timestamp"],weather_2019["temperature"])
ax.set_xlabel("Date")
ax.set_ylabel("Grad")
ax.set_title("Temperature in Sanfrancisco in 2019")
plt.show()


Our goal is to find out how would the weather affect the bike rent business. With the help of idle_time could we analyze under what kinf of weather (e.g Temperature, Windspeed) where should we put our bikes, so that they can be rented as frequently as possible. We assume that normally when the temparature is high, people would go out like beach, and when it's cold, people would spend time in city center like shopping mall. Then we can separate the weather data in seasons to have a better look.

In [ ]:
# create month feature
weather_2019["Month"] = weather_2019["timestamp"].apply(lambda dt: dt.month)
weather_2019.head()

In [ ]:
# create four seasons
spring_month=[3,4,5]
spring_2019 = weather_2019[weather_2019["Month"].isin(spring_month)==True]

summer_month=[6,7,8]
summer_2019 = weather_2019[weather_2019["Month"].isin(summer_month)==True]

autumn_month=[9,10,11]
autumn_2019 = weather_2019[weather_2019["Month"].isin(autumn_month)==True]

winter_month=[12,1,2]
winter_2019 = weather_2019[weather_2019["Month"].isin(winter_month)==True]

summer_2019.head(10)